Test for new dynamics where only one state transition at a time. 

In [1]:
# %load_ext autoreload
# %autoreload 2
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics, new_dynamics2
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)
from numba import njit
from scripts.functions import *


In [2]:
length=[12, 12]
V = 0.9
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 2, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)

# ma._ws[:] = 0
# ma._set_bare_parameters(
#     ma._a, ma._b, ma._w, ma._as, ma._bs, ma._ws, ma._autom, ma._z2
# )
ma.load(f"/home/keisuke/Documents/Research/sshfs/DimerMaster/save/ma/h={h}V={V}l={length}")

yoyo
2


In [3]:
n_jobs = 12

g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)

op = f.dimer_hamiltonian(h, V,np.array(length))
op_transition = f.dimer_flip1(length = np.array(length))
hex_ = nk.machine.new_hex(np.array(length))




sweep_size = 1000
sa_mul = nk.sampler.DimerMetropolisLocal_multi(machine=ma, op=op_transition, length = length, n_chains=1, sweep_size = sweep_size, kernel = 1, n_jobs=n_jobs)
sa_mul.reset()
_ = sa_mul.generate_samples(12) # discard the begginings of metropolis sampling.

yoyo
number of core : 12


In [4]:
n_samples_ = 12
samples_state = sa_mul.generate_samples(int(n_samples_ / 1))
samples_state = samples_state.reshape(-1, ma.hilbert.size)
print('get samples')

get samples


In [138]:
import random
class new_dynamics4(new_dynamics):
    
    
    def dynamics(self, X, t_list):
        
        # assert X.dtype == np.int8
        
        assert isinstance(t_list, np.ndarray)
        assert t_list.ndim == 1
        

        P = self._dynamics(
                X, 
                t_list,
                self.basis,
                self.constant,
                self.diag_mels,
                self.n_conns,
                self.mels,
                self.x_prime,
                self.acting_on,
                self._w,
                self.c_r,
                self.s_r
        )

        return P

    
    
    @staticmethod
#     @njit
    def _dynamics(
            X,
            t_list,
            _basis,
            _constant,
            _diag_mels,
            _n_conns,
            _mels,
            _x_prime,
            _acting_on,
            _w,
            c_r,
            s_r,
            ):
        
        # basis is float64[:]
        

        batch_size = X.shape[0]
        t_num = len(t_list)
        L = X.shape[1]
        p_array = np.zeros((batch_size, t_num, L),dtype= np.int8) 
        p_array[:,0,:] = X[:,:]
        X = X.astype(np.int8)
        _x_prime = _x_prime.astype(np.int8)
        
        transition_X = np.zeros((10**3,L),dtype=np.int8 )

        a_0 = 0
#         T = np.zeros(batch_size)
        t_delta = t_list[1] - t_list[0]


        for b in range(batch_size):
            l = 0
            T = 0
            ti_old = np.int64(0)
            x_b = X[b]
            P = p_array[b]
            sections = 0
            
            while True:


                r = (x_b).astype(np.float64).dot(_w)
                tan = np.tanh(r)
#                 print(tan)



                x_prime, sites, mels  = get_transition_one(
                                    x_b,
                                    c_r,
                                    s_r,
                                    tan,
                                    sections,
                                    _basis,
                                    _constant,
                                    _diag_mels,
                                    _n_conns,
                                    _mels,
                                    _x_prime,
                                    _acting_on)

            
                N_conn = mels.shape[0]
#                 print(x_b)
#                 print(x_prime, sites, mels)
#                 for n in range(batch_size):
#                     a_0[n] = (-1)* mels[sections[n]: sections[n+1]].sum()
                a_0 = (-1) * mels.sum()
                print(a_0)
    
#                 if (l<10**3) & (a_0 > 300):
#                     transition_X[l] = x_b
#                     l+=1
#                     for i in range(x_prime.shape[0]):
#                         if l<10**3:
#                             transition_X[l] = x_b
#                             transition_X[l][sites[i]] = x_prime[i]
#                             l+=1
#                     l+=1
    
#                 print(mels)
                r_1 = random.random()
                r_2 = random.random()

                tau = np.log(1/r_1)/a_0
                T += tau

#                 print(T, x_b)
                if T > (t_list[-1] + t_delta):
                    P[ti_old+1:] = x_b
                    break
                
                ti_new = np.int64(T // t_delta)

                if ti_new > ti_old:
                    P[ti_old+1:ti_new+1] = x_b
#                     print(P)
#                     P[ti_new] = x_b
                    ti_old = ti_new
                s = 0
                for i in range(N_conn):
                    s -= mels[i]
                    if s >= r_2 * a_0:
                        x_b[sites[i]] = x_prime[i]
                        break
            p_array[b,:,:] = P[:,:]
            print('iteration : ',l)

        return p_array
            
        
    def run(self, X, t_list, qout):
        
        out = self.dynamics(X, t_list)
        
        qout.put(out)
    
        
    def multiprocess(self, X, t_list,  n = 1):
        queue = []
        process = []
        N = X.shape[0] 
        index = np.round(np.linspace(0, N, n + 1)).astype(np.int)

        for i in range(n):
            queue.append(mp.Queue())
            p = mp.Process(target=self.run, args=(X[index[i]:index[i+1]], t_list ,queue[i]))
            p.start()
            process.append(p)
        
        out1 = []
        for q in queue:
            out = q.get()
            out1.append(out)
        
        return np.concatenate(out1, axis=1)

In [141]:
# @njit
def get_transition_one(
        x,
        c_r,
        s_r,
        tan,
        sections,
        basis,
        constant,
        diag_mels,
        n_conns,
        all_mels,
        all_x_prime,
        acting_on
    ):


    n_sites = x.shape[0]


    n_operators = n_conns.shape[0]
    xs_n = np.empty(n_operators, dtype=np.intp)
    xs_n_prime = np.int64(0)
    max_conn = 0
    batch_size = 1

    tot_conn = 0

    sections = 1
    
    for i in range(n_operators):
        n_conns_i = n_conns[i]
        x_i = (x[acting_on[i]] + 1) / 2
        s = 0
        for j in range(4):
            s += x_i[j] * basis[j]
        xs_n[i] = s
        sections += n_conns_i[xs_n[i]]
#         print(xs_n[i],n_conns_i[xs_n[i]])
    
    sections -= 1
    tot_conn=sections 




    x_prime = np.empty((tot_conn, 2), dtype=np.int8) # x.shpae[0] is number of connected elements of hamiltonian from batch of states. 
    t_mels = np.empty(tot_conn, dtype=np.float64)
    sites_ = np.empty((tot_conn, 2), dtype=np.int16)
    
    
    acting_on = acting_on[:,1:3].copy()
    
    basis_r = np.array([3,1])
    c = 0
    
    
    x_batch = x
    xs_n_b = xs_n
    tan_b = tan


    for i in range(n_operators):



        n_conn_i = n_conns[i, xs_n_b[i]]

        if n_conn_i > 0:
            sites = acting_on[i]
            s_r_i = s_r[i]
            c_r_i = c_r[i]
            for cc in range(n_conn_i):

                x_prime_cc = x_prime[c + cc]
                x_prime_cc[:] = all_x_prime[i, xs_n_b[i], cc]
                sites_[c + cc] = sites

                num = ((np.sum((x_prime_cc - x_batch[sites]) * basis_r) + 8)/2)

                sin_prime = s_r_i[np.int(num)]
                cos_prime = c_r_i[np.int(num)]


                t_mels[c + cc] = all_mels[i, xs_n_b[i], cc] *np.prod(tan_b*sin_prime + cos_prime)
            c += n_conn_i

#     print(x_prime.shape)

    return x_prime, sites_, t_mels

In [143]:
t_list = np.arange(0,5,0.1)
d = new_dynamics4(op, ma)
%time P_= d.dynamics(samples_state[0].reshape(1,-1), t_list) 

41.1598932560075
40.3384636283029
38.640568381412166
38.599020469055496
38.63114019305528
40.188644550555544
40.98263291582862
39.350940898798655
40.16140993224052
40.151349407639465
40.12318920149256
39.341342304845085
39.253305764947044
38.43649807297981
39.2868056528284
39.259073137171306
40.09900629556643
41.66613137235178
40.94210172009901
42.600338857214325
41.77982281371388
41.817562991219546
42.59886894457054
43.44125112080518
43.42431480766753
42.58045198944225
42.59973431922476
41.819267452418266
40.20307769765709
40.215808174369265
41.144962390015685
40.286148891981995
40.288758204509
40.29842957055957
38.8155453382979
40.37812778673865
41.13749735153374
40.24639259903129
41.920387211934575
41.076419411752816
41.05397193561926
41.91793858069768
40.99992369212681
41.79997336681104
40.95534269001322
40.07324594737148
40.19024110060695
41.022514011590715
40.27338298332819
39.483912185325664
41.01837828107679
41.80988442641419
40.29673957459974
41.093530826334565
40.334442076092

In [117]:
np.where(op._acting_on < 0)

(array([], dtype=int64), array([], dtype=int64))

In [121]:
from scripts.dynamics import get_transition
sections = 0
x_b = state
r = (x_b).astype(np.float64).dot(d._w)
tan = np.tanh(r)

x_prime, sites, mels  = get_transition_1(
                    x_b,
                    d.c_r,
                    d.s_r,
                    tan,
                    sections,
                    d.basis,
                    d.constant,
                    d.diag_mels,
                    d.n_conns,
                    d.mels,
                    d.x_prime,
                    d.acting_on)

# x_prime2, sites2, mels2  = get_transition(
#                     x_b.reshape(1,-1),
#                     d.c_r,
#                     d.s_r,
#                     tan.reshape(1,-1),
#                     np.zeros(1, dtype = np.int64),
#                     d.basis,
#                     d.constant,
#                     d.diag_mels,
#                     d.n_conns,
#                     d.mels,
#                     d.x_prime,
#                     d.acting_on)

9 0
0 0
0 0
9 0
3 0
12 0
6 0
15 0
15 0
6 0
12 0
3 0
8 0
0 0
0 0
1 0
0 0
0 0
1 1
0 0
0 0
8 0
3 0
12 0
7 0
15 0
15 0
6 0
12 0
11 0
11 0
12 0
9 1
9 0
3 0
13 0
7 0
15 0
15 0
6 0
12 0
11 0
11 0
12 0
8 0
9 0
2 0
12 0
7 0
14 0
12 0
10 0
10 1
12 0
7 0
14 0
12 0
10 0
10 1
12 0
7 0
14 0
12 0
10 0
11 0
12 0
6 0
15 0
14 0
6 0
12 0
2 0
8 0
8 0
3 0
12 0
6 1
15 0
14 0
14 0
12 0
3 0
8 0
0 0
0 0
0 0
0 0
0 0
1 0
0 0
1 0
1 0
3 0
12 0
7 0
15 0
14 0
15 0
15 0
15 0
15 0
7 0
13 0
3 0
7 0
15 0
15 0
15 0
14 0
15 0
15 0
7 0
12 0
3 0
9 0
0 0
0 0
1 0
0 0
0 0
1 0
8 0
1 0
0 0
0 0
1 0
1 0
8 0
0 0
0 0
0 0
0 0
1 0
8 0
1 0
0 0
0 0
9 1
2 0
12 0
8 0
0 0
1 0
0 0
14 0
14 0
15 0
6 0
12 0
3 0
8 0
8 0
0 0
1 0
0 0
0 0
0 0
0 0
0 0
8 0
2 0
12 0
7 0
6 1
12 0
10 0
10 1
12 0
7 0
14 0
12 0
10 0
10 1
12 0
7 0
14 0
12 0
10 0
11 0
12 0
6 0
15 0
15 0
6 0
12 0
3 0
8 0
0 0
1 0
9 0
3 0
5 1
4 0
3 0
7 0
6 0
12 0
3 0
8 0
0 0
1 0
9 0
3 0
5 1
4 0
3 0
7 0
6 0
12 0
3 0
8 0
0 0
0 0
9 0
3 0
12 0
7 0
15 0
15 0
7 0
12 0
3 0
8 1
0 0
0 0
1 0
0 0
0 0
1 

In [122]:
test = np.zeros((x_prime.shape[0],x_b.shape[0]))
test[0] = x_b
for i in range(test.shape[0]-1):
    test[i+1] = x_b
    test[i+1][sites[i]] = x_prime[i]
    

In [123]:
# from importlib import reload
# reload(nk)
# reload(f)
op = f.dimer_hamiltonian(h, V,np.array(length))
output_get_conn = op.get_conn(x_b)[0]

yoyo
xs_n 9  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 9  n_conn 0
xs_n 3  n_conn 0
xs_n 12  n_conn 0
xs_n 6  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 6  n_conn 0
xs_n 12  n_conn 0
xs_n 3  n_conn 0
xs_n 8  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 1  n_conn 0
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 1  n_conn 1
xs_n 0  n_conn 0
xs_n 0  n_conn 0
xs_n 8  n_conn 0
xs_n 3  n_conn 0
xs_n 12  n_conn 0
xs_n 7  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 6  n_conn 0
xs_n 12  n_conn 0
xs_n 11  n_conn 0
xs_n 11  n_conn 0
xs_n 12  n_conn 0
xs_n 9  n_conn 1
xs_n 9  n_conn 0
xs_n 3  n_conn 0
xs_n 13  n_conn 0
xs_n 7  n_conn 0
xs_n 15  n_conn 0
xs_n 15  n_conn 0
xs_n 6  n_conn 0
xs_n 12  n_conn 0
xs_n 11  n_conn 0
xs_n 11  n_conn 0
xs_n 12  n_conn 0
xs_n 8  n_conn 0
xs_n 9  n_conn 0
xs_n 2  n_conn 0
xs_n 12  n_conn 0
xs_n 7  n_conn 0
xs_n 14  n_conn 0
xs_n 12  n_conn 0
xs_n 10  n_conn 0
xs_n 10  n_conn 1
xs_n 12  n_conn 0
xs_n 7  n_conn 0
xs_n 14  n_conn 0
xs_n 12  n_conn 0


In [124]:
hex_.is_dimer_basis(test)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [23]:
hex_.is_dimer_basis(output_get_conn)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [ ]:
hex_.is_dimer_basis(output_get_conn)

In [80]:
edges = hex_.for_hamiltonian()[0]

In [ ]:
length=[4, 4]
V = 1.0
h = 1.0
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)


ham = f.dimer_hamiltonian(V = V, h = h ,length=np.array(length))
op_transition1 = f.dimer_flip1(length = np.array(length))


hex_ = nk.machine.new_hex(np.array(length))

ma = nk.machine.RbmDimer(hi, hex_, alpha = 1, symmetry = True
                    ,use_hidden_bias = False, use_visible_bias = False, dtype=float, reverse=True, half=True)
ma.init_random_parameters(seed=1234)